In [1]:
import pandas as pd
import numpy as np
import os
print(os.getcwd())
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s [%(levelname)s] %(message)s',
                    level=logging.INFO)
from datetime import datetime, timedelta
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
import gc

D:\notebook\ALi


In [2]:
# data_path
train_pre_sel_log_path1 = "data/preliminary_sel_log_dataset.csv"
train_pre_sel_log_path2 = "data/preliminary_sel_log_dataset_a.csv"
train_label_path1 = "data/preliminary_train_label_dataset.csv"
train_label_path2 = "data/preliminary_train_label_dataset_s.csv"
test_submit_path = "data/preliminary_submit_dataset_a.csv"


标签数据EDA

In [3]:
logger.info("label EDA")
train_label1 = pd.read_csv(train_label_path1, parse_dates=["fault_time"])
train_label2 = pd.read_csv(train_label_path2, parse_dates=["fault_time"])
train_label = pd.concat([train_label1,train_label2 ])
train_label.reset_index(drop=True, inplace=True)

2022-04-05 12:04:34,519 [INFO] label EDA


In [4]:
print(train_label.head())

print(f"num reconds :{len(train_label)}")
# print(f"num server:{len(set(train_label))}")
print("label count:")
print(train_label["label"].value_counts())
print("server count")
print(train_label["sn"].value_counts())

             sn          fault_time  label
0  SERVER_25698 2020-10-09 13:43:00      0
1  SERVER_25699 2020-08-25 18:50:00      0
2  SERVER_25712 2020-03-16 13:20:00      0
3  SERVER_25708 2020-07-25 12:44:00      0
4  SERVER_25711 2020-03-16 16:51:00      0
num reconds :16669
label count:
2    9343
1    3387
3    2463
0    1476
Name: label, dtype: int64
server count
SERVER_13884    8
SERVER_12517    8
SERVER_23459    8
SERVER_23217    7
SERVER_3352     6
               ..
SERVER_24585    1
SERVER_22492    1
SERVER_1876     1
SERVER_6540     1
SERVER_16807    1
Name: sn, Length: 13705, dtype: int64


基础日志数据EDA

In [5]:
train_pre_sel_log1 = pd.read_csv(train_pre_sel_log_path1, parse_dates=["time"])
train_pre_sel_log2 = pd.read_csv(train_pre_sel_log_path2, parse_dates=["time"])
train_pre_sel_log = pd.concat([train_pre_sel_log1, train_pre_sel_log2])

print(train_pre_sel_log.head())
print(len(train_pre_sel_log))
print(train_pre_sel_log["sn"].value_counts())
print(max(train_pre_sel_log["time"]))
print(min(train_pre_sel_log["time"]))
print("msg")
print(train_pre_sel_log["msg"].value_counts())

             sn                time  \
0  SERVER_25698 2020-10-09 08:32:21   
1  SERVER_25698 2020-10-09 07:43:48   
2  SERVER_25698 2020-10-09 08:16:22   
3  SERVER_25698 2020-10-09 05:46:41   
4  SERVER_25698 2020-10-09 12:59:13   

                                                 msg server_model  
0   System Boot Initiated BIOS_Boot_Up | State As...          SM0  
1   System Boot Initiated BIOS_Boot_Up | State As...          SM0  
2   System Boot Initiated BIOS_Boot_Up | State As...          SM0  
3   System Boot Initiated BIOS_Boot_Up | State As...          SM0  
4   System Boot Initiated BIOS_Boot_Up | State As...          SM0  
493527
SERVER_26689    6958
SERVER_23498    6821
SERVER_9056     6049
SERVER_4846     5519
SERVER_22927    5495
                ... 
SERVER_26248       1
SERVER_6230        1
SERVER_26936       1
SERVER_7876        1
SERVER_10611       1
Name: sn, Length: 16588, dtype: int64
2020-11-25 23:21:06
2019-12-27 23:38:05
msg
 Memory #0xf9 | Correctable ECC | Ass

In [6]:
test_submit = pd.read_csv(test_submit_path, parse_dates=["fault_time"])
test_submit["fault_time_label"] = test_submit["fault_time"].dt.floor(freq="5T")
print(f"num submit:{len(test_submit)}")
print(test_submit["sn"].value_counts())
print(test_submit.head())

num submit:3011
330d7e1153c4    3
eec3d17b0128    3
16a8dff8cc19    3
151ff729ef36    3
a49f03aa6bd9    3
               ..
cf9935ff3b8a    1
56f1b9334e5f    1
6d943794e7c9    1
b4c04c3f47cb    1
940e1f79c5a5    1
Name: sn, Length: 2883, dtype: int64
             sn          fault_time    fault_time_label
0  000d33b21436 2020-09-02 16:42:54 2020-09-02 16:40:00
1  005c5a9218ba 2020-06-28 19:05:16 2020-06-28 19:05:00
2  0079283bde6e 2020-04-26 21:32:44 2020-04-26 21:30:00
3  007bdf23b62f 2020-06-16 18:40:39 2020-06-16 18:40:00
4  00a577a8e54f 2020-04-07 07:16:55 2020-04-07 07:15:00


数据处理

In [7]:
train_pre_sel_log["time_label"] = train_pre_sel_log["time"].dt.floor(freq="5T")
train_pre_sel_log = train_pre_sel_log.groupby(["sn", "time_label"]).agg({"msg":list}).reset_index(drop=False)
train_pre_sel_log["msg"] = train_pre_sel_log["msg"].apply(lambda x: ','.join(x))


分词、训练

In [13]:
embedding_size = 16
msg_list = list(train_pre_sel_log["msg"])
tokenized_sent = [word_tokenize(s.lower()) for s in msg_list] # 分词
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_sent)]
model = Doc2Vec(tagged_data, vector_size = embedding_size, window = 2, min_count = 1, epochs = 10)
embedding_vec = []
for i in tqdm(range(len(msg_list))):
    embedding_vec.append(model.infer_vector(word_tokenize(" ".join(msg_list[i]))))

embedding_vec = np.array(embedding_vec)

for i in tqdm(range(embedding_size)):
    train_pre_sel_log[f"embedding_feat{i}"] = embedding_vec[:, i]

print(train_pre_sel_log.head())

del embedding_vec;gc.collect()


2022-04-05 12:13:16,815 [INFO] collecting all words and their counts
2022-04-05 12:13:16,815 [INFO] PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2022-04-05 12:13:16,869 [INFO] PROGRESS: at example #10000, processed 331327 words (6206345/s), 644 word types, 10000 tags
2022-04-05 12:13:16,959 [INFO] PROGRESS: at example #20000, processed 935124 words (6770398/s), 755 word types, 20000 tags
2022-04-05 12:13:17,045 [INFO] PROGRESS: at example #30000, processed 1499399 words (6680973/s), 881 word types, 30000 tags
2022-04-05 12:13:17,122 [INFO] PROGRESS: at example #40000, processed 1983756 words (6348318/s), 1069 word types, 40000 tags
2022-04-05 12:13:17,253 [INFO] PROGRESS: at example #50000, processed 2881773 words (6915318/s), 1442 word types, 50000 tags
2022-04-05 12:13:17,354 [INFO] PROGRESS: at example #60000, processed 3530870 words (6462895/s), 1766 word types, 60000 tags
2022-04-05 12:13:17,430 [INFO] PROGRESS: at example #70000, processed 3998908 words 

2022-04-05 12:14:03,397 [INFO] worker thread finished; awaiting finish of 1 more threads
2022-04-05 12:14:03,399 [INFO] worker thread finished; awaiting finish of 0 more threads
2022-04-05 12:14:03,399 [INFO] EPOCH - 9 : training on 4640913 raw words (1796958 effective words) took 3.0s, 591952 effective words/s
2022-04-05 12:14:04,418 [INFO] EPOCH 10 - PROGRESS: at 33.89% examples, 457693 words/s, in_qsize 5, out_qsize 0
2022-04-05 12:14:05,423 [INFO] EPOCH 10 - PROGRESS: at 67.75% examples, 600012 words/s, in_qsize 5, out_qsize 0
2022-04-05 12:14:06,442 [INFO] EPOCH 10 - PROGRESS: at 98.64% examples, 588540 words/s, in_qsize 2, out_qsize 1
2022-04-05 12:14:06,444 [INFO] worker thread finished; awaiting finish of 2 more threads
2022-04-05 12:14:06,446 [INFO] worker thread finished; awaiting finish of 1 more threads
2022-04-05 12:14:06,447 [INFO] worker thread finished; awaiting finish of 0 more threads
2022-04-05 12:14:06,447 [INFO] EPOCH - 10 : training on 4640913 raw words (1797438 e

             sn          time_label  \
0  000d33b21436 2020-09-02 11:35:00   
1  000d33b21436 2020-09-02 15:45:00   
2  005c5a9218ba 2020-06-28 18:25:00   
3  005c5a9218ba 2020-06-28 18:35:00   
4  005c5a9218ba 2020-06-28 18:40:00   

                                                 msg  embedding_feat0  \
0   System Boot Initiated BIOS_Boot_Up | Initiate...        -0.458311   
1   System Boot Initiated BIOS_Boot_Up | Initiate...        -0.417290   
2   Memory Memory_Status | Correctable ECC | Asse...         0.192492   
3   System ACPI Power State #0x7d | S4/S5: soft-o...         0.106979   
4   System ACPI Power State #0x7d | S0/G0: workin...        -0.149938   

   embedding_feat1  embedding_feat2  embedding_feat3  embedding_feat4  \
0         0.807872        -0.180104         0.016189         0.981122   
1         0.714134        -0.209261         0.042042         0.882467   
2         1.652067         0.949054        -0.815698         1.369072   
3         0.416753        -0.26396

0

In [24]:
train_label

,sn,fault_time,label,fault_time_label
0,SERVER_25698,2020-10-09 13:43:00,0,2020-10-09 13:40:00
1,SERVER_25699,2020-08-25 18:50:00,0,2020-08-25 18:50:00
2,SERVER_25712,2020-03-16 13:20:00,0,2020-03-16 13:20:00
3,SERVER_25708,2020-07-25 12:44:00,0,2020-07-25 12:40:00
4,SERVER_25711,2020-03-16 16:51:00,0,2020-03-16 16:50:00
...,...,...,...,...
16664,SERVER_24971,2020-03-04 21:09:00,3,2020-03-04 21:05:00
16665,SERVER_24971,2020-11-12 20:49:00,3,2020-11-12 20:45:00
16666,SERVER_24962,2020-09-12 12:18:00,3,2020-09-12 12:15:00
16667,SERVER_24971,2020-10-04 17:41:00,3,2020-10-04 17:40:00


获取数据集label

In [14]:
train_label["fault_time_label"] = train_label["fault_time"].dt.floor(freq="5T")
print(train_label.head())
print(train_label.groupby(["sn", "fault_time_label"]).count())

             sn          fault_time  label    fault_time_label
0  SERVER_25698 2020-10-09 13:43:00      0 2020-10-09 13:40:00
1  SERVER_25699 2020-08-25 18:50:00      0 2020-08-25 18:50:00
2  SERVER_25712 2020-03-16 13:20:00      0 2020-03-16 13:20:00
3  SERVER_25708 2020-07-25 12:44:00      0 2020-07-25 12:40:00
4  SERVER_25711 2020-03-16 16:51:00      0 2020-03-16 16:50:00
                                  fault_time  label
sn           fault_time_label                      
SERVER_10001 2020-05-01 10:00:00           1      1
SERVER_10003 2020-03-28 09:45:00           1      1
SERVER_10008 2020-02-25 16:10:00           1      1
             2020-03-11 18:00:00           1      1
SERVER_10009 2020-05-08 16:35:00           1      1
...                                      ...    ...
SERVER_9991  2020-08-04 22:45:00           1      1
             2020-10-07 18:40:00           1      1
SERVER_9993  2020-05-14 23:50:00           1      1
SERVER_9998  2020-05-29 11:25:00           1      

训练集数据：特征为前一天所有的日志embedding均值

In [15]:

train_label.reset_index(drop=True, inplace=True)
feat_cols = [f"embedding_feat{i}" for i in range(embedding_size)]


data = pd.DataFrame()
for i in tqdm(range(len(train_label))):
    sn_i = train_label.loc[i, "sn"]
    fault_time = train_label.loc[i, "fault_time_label"]
    start_time = fault_time - timedelta(days=1)
    df_ = train_pre_sel_log[
        (train_pre_sel_log["sn"] == sn_i) & (train_pre_sel_log["time_label"] < fault_time) & (train_pre_sel_log["time_label"] >= start_time)]
    df_ = df_.groupby("sn")[feat_cols].mean().reset_index(drop=False)
    df_["fault_time_label"] = fault_time
    data = data.append(df_)
        
# 特征标签合并
logger.info(f"{len(data)}, {len(train_label)}")
data = pd.merge(data, train_label, on=["sn", "fault_time_label"], how="inner")
        
print(len(data))
print(data.head())

100%|███████████████████████████████████████████████████████████████████████████| 16669/16669 [01:47<00:00, 155.09it/s]
2022-04-05 12:20:52,836 [INFO] 15787, 16669


16165
             sn  embedding_feat0  embedding_feat1  embedding_feat2  \
0  SERVER_25698        -0.391262         0.819642        -0.156425   
1  SERVER_25699         0.773447         2.193513         0.453427   
2  SERVER_25712         1.213543         3.250009         0.116699   
3  SERVER_25708         1.192098         2.515469        -0.734996   
4  SERVER_25711         1.314781         2.883857         0.506901   

   embedding_feat3  embedding_feat4  embedding_feat5  embedding_feat6  \
0         0.032034         0.996022        -0.505743        -0.037747   
1        -2.092675         2.294552        -0.096203        -1.964372   
2        -3.003966         3.331649        -0.049078        -3.197646   
3        -1.666027         2.932497        -0.368600        -2.680381   
4        -2.122538         2.954514        -0.692048        -2.434330   

   embedding_feat7  embedding_feat8  embedding_feat9  embedding_feat10  \
0        -0.086605         0.103272         0.251918        

In [25]:
data

,sn,embedding_feat0,embedding_feat1,embedding_feat2,embedding_feat3,embedding_feat4,embedding_feat5,embedding_feat6,embedding_feat7,embedding_feat8,embedding_feat9,embedding_feat10,embedding_feat11,embedding_feat12,embedding_feat13,embedding_feat14,embedding_feat15,fault_time_label,fault_time,label
0,SERVER_25698,-0.391262,0.819642,-0.156425,0.032034,0.996022,-0.505743,-0.037747,-0.086605,0.103272,0.251918,-0.022964,-0.155762,-0.861019,0.047662,0.389933,-0.619980,2020-10-09 13:40:00,2020-10-09 13:43:00,0
1,SERVER_25699,0.773447,2.193513,0.453427,-2.092675,2.294552,-0.096203,-1.964372,0.211137,-1.985654,-0.678864,-0.784672,1.183363,-1.099988,0.664408,0.762391,0.895570,2020-08-25 18:50:00,2020-08-25 18:50:00,0
2,SERVER_25712,1.213543,3.250009,0.116699,-3.003966,3.331649,-0.049078,-3.197646,0.065529,-3.108294,-1.196077,-1.194262,2.112895,-1.317445,1.072312,0.974382,1.784223,2020-03-16 13:20:00,2020-03-16 13:20:00,0
3,SERVER_25708,1.192098,2.515469,-0.734996,-1.666027,2.932497,-0.368600,-2.680381,-0.293736,-2.336942,-0.832494,-0.459026,1.045525,-1.132936,1.337776,1.362077,1.008783,2020-07-25 12:40:00,2020-07-25 12:44:00,0
4,SERVER_25711,1.314781,2.883857,0.506901,-2.122538,2.954514,-0.692048,-2.434330,0.449839,-2.206861,-0.861978,-1.319967,1.643967,-1.449167,1.182395,0.909759,1.974011,2020-03-16 16:50:00,2020-03-16 16:51:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16160,SERVER_24971,-0.109670,0.543262,-0.010573,-0.086476,0.568227,-0.321074,-0.106547,0.013405,0.031380,0.207169,-0.156012,-0.028538,-0.378417,0.161626,0.193726,-0.103938,2020-03-04 21:05:00,2020-03-04 21:09:00,3
16161,SERVER_24971,-0.080201,0.684935,-0.057636,-0.218827,0.668698,-0.346357,-0.154062,0.083343,-0.093313,0.100968,-0.198593,0.001740,-0.521748,0.034694,0.294387,-0.170491,2020-11-12 20:45:00,2020-11-12 20:49:00,3
16162,SERVER_24962,-0.113262,0.440764,-0.266011,0.376740,0.848466,-0.580546,0.033483,-0.046838,0.416968,0.400400,0.090399,-0.399645,-0.678022,0.107308,0.608970,-0.423980,2020-09-12 12:15:00,2020-09-12 12:18:00,3
16163,SERVER_24971,-0.120981,0.614469,-0.023611,-0.111894,0.665326,-0.397653,-0.028675,0.014909,0.035518,0.162600,-0.148406,-0.048257,-0.553547,-0.001327,0.280442,-0.143964,2020-10-04 17:40:00,2020-10-04 17:41:00,3


预测集数据

In [16]:
test = pd.DataFrame()
for i in tqdm(range(len(test_submit))):
    sn_i = test_submit.loc[i, "sn"]
    fault_time = test_submit.loc[i, "fault_time_label"]
    start_time = fault_time - timedelta(days=1)
    df_ = train_pre_sel_log[
        (train_pre_sel_log["sn"] == sn_i) & (train_pre_sel_log["time_label"] < fault_time) & (train_pre_sel_log["time_label"] >= start_time)]
    df_ = df_.groupby("sn")[feat_cols].mean().reset_index(drop=False)
    df_["fault_time_label"] = fault_time
    test = test.append(df_)
        
test.drop_duplicates(keep="first", inplace=True)
# 特征标签合并
logger.info(f"{len(test)}, {len(test_submit)}")
test = pd.merge(test, test_submit, on=["sn", "fault_time_label"], how="right")
        
print(len(test))
print(test_submit.head())
    


100%|█████████████████████████████████████████████████████████████████████████████| 3011/3011 [00:20<00:00, 146.43it/s]
2022-04-05 12:27:00,713 [INFO] 2790, 3011


3011
             sn          fault_time    fault_time_label
0  000d33b21436 2020-09-02 16:42:54 2020-09-02 16:40:00
1  005c5a9218ba 2020-06-28 19:05:16 2020-06-28 19:05:00
2  0079283bde6e 2020-04-26 21:32:44 2020-04-26 21:30:00
3  007bdf23b62f 2020-06-16 18:40:39 2020-06-16 18:40:00
4  00a577a8e54f 2020-04-07 07:16:55 2020-04-07 07:15:00


In [26]:
test

,sn,embedding_feat0,embedding_feat1,embedding_feat2,embedding_feat3,embedding_feat4,embedding_feat5,embedding_feat6,embedding_feat7,embedding_feat8,embedding_feat9,embedding_feat10,embedding_feat11,embedding_feat12,embedding_feat13,embedding_feat14,embedding_feat15,fault_time_label,fault_time
0,000d33b21436,-0.437800,0.761003,-0.194682,0.029115,0.931794,-0.441966,-0.031078,-0.094967,0.138500,0.288345,0.006819,-0.122875,-0.835054,-0.043942,0.429964,-0.671537,2020-09-02 16:40:00,2020-09-02 16:42:54
1,005c5a9218ba,0.049844,0.989157,0.157853,-0.158190,1.135755,-0.711239,-0.183801,0.200195,-0.232013,0.110504,-0.340430,0.069531,-0.873945,0.307114,0.236544,-0.106463,2020-06-28 19:05:00,2020-06-28 19:05:16
2,0079283bde6e,-0.192264,0.690155,0.066562,-0.286142,0.631531,-0.297805,-0.078585,0.011457,-0.188678,0.124595,-0.140251,0.058425,-0.617268,0.010791,0.195086,-0.142621,2020-04-26 21:30:00,2020-04-26 21:32:44
3,007bdf23b62f,0.081063,0.890372,-0.153870,-0.664797,1.130413,-0.005183,-0.673220,0.004616,-0.769474,-0.137612,-0.278883,0.222832,-0.372111,0.131853,0.373560,-0.252677,2020-06-16 18:40:00,2020-06-16 18:40:39
4,00a577a8e54f,-0.016810,0.862955,0.257293,-0.294535,0.830550,-0.661618,-0.021108,0.157812,-0.107809,0.182205,-0.280013,0.153066,-0.795197,0.060899,0.197124,0.039337,2020-04-07 07:15:00,2020-04-07 07:16:55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006,fca123853b8a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-23 19:20:00,2020-09-23 19:21:23
3007,fcb96d7ac147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-15 04:50:00,2020-07-15 04:54:07
3008,fe039115754e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-28 03:10:00,2020-08-28 03:12:40
3009,fe2edf985065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-06 17:50:00,2020-09-06 17:54:31


In [17]:
del train_pre_sel_log;gc.collect()

20

In [18]:
folds = StratifiedKFold(n_splits=5,shuffle=True, random_state=2022)
test_x = test[feat_cols]
test_y = np.zeros((len(test_x), 4),np.float16)
for i, (train_idx, valid_idx) in enumerate(folds.split(data[feat_cols], data["label"])):
    logger.info(f"#####training fold{i}#####")
    train_x, valid_x = data.loc[train_idx, feat_cols], data.loc[valid_idx, feat_cols]
    train_y, valid_y = data.loc[train_idx, "label"], data.loc[valid_idx, "label"]
    print(train_x.shape, valid_x.shape)
    clf = XGBClassifier(
        learning_rate=0.1, 
        n_estimators=100,
        gamma=0.001, 
        subsample=0.8, 
        colsample_bytree=0.8,
        reg_lambda=1, 
        reg_alpha=1,
        objective="multi:softmax"
    )
    clf.fit(train_x, train_y, 
            eval_set=[(train_x, train_y), (valid_x, valid_y)],
            eval_metric=["merror", "mlogloss"]
            , early_stopping_rounds=30, verbose=10)
    test_y += clf.predict_proba(test_x)
    
pred_y = np.argmax(test_y, axis=1)

test_submit["label"] = pred_y
df_submit = test_submit[["sn", "fault_time", "label"]]

2022-04-05 12:27:09,748 [INFO] #####training fold0#####


(12932, 16) (3233, 16)
[0]	validation_0-merror:0.34248	validation_0-mlogloss:1.32190	validation_1-merror:0.36158	validation_1-mlogloss:1.32503


D:\anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10]	validation_0-merror:0.32795	validation_0-mlogloss:0.99861	validation_1-merror:0.34890	validation_1-mlogloss:1.03470
[20]	validation_0-merror:0.30622	validation_0-mlogloss:0.86844	validation_1-merror:0.34086	validation_1-mlogloss:0.93516
[30]	validation_0-merror:0.28967	validation_0-mlogloss:0.79552	validation_1-merror:0.33096	validation_1-mlogloss:0.88869
[40]	validation_0-merror:0.27351	validation_0-mlogloss:0.74407	validation_1-merror:0.32292	validation_1-mlogloss:0.86471
[50]	validation_0-merror:0.26214	validation_0-mlogloss:0.70705	validation_1-merror:0.31828	validation_1-mlogloss:0.84901
[60]	validation_0-merror:0.25039	validation_0-mlogloss:0.67805	validation_1-merror:0.31519	validation_1-mlogloss:0.83887
[70]	validation_0-merror:0.23840	validation_0-mlogloss:0.64999	validation_1-merror:0.31302	validation_1-mlogloss:0.83124
[80]	validation_0-merror:0.22556	validation_0-mlogloss:0.62249	validation_1-merror:0.31148	validation_1-mlogloss:0.82421
[90]	validation_0-merror:0.21513

2022-04-05 12:27:12,057 [INFO] #####training fold1#####


...

(12932, 16) (3233, 16)
[0]	validation_0-merror:0.34411	validation_0-mlogloss:1.32205	validation_1-merror:0.38695	validation_1-mlogloss:1.32761
[10]	validation_0-merror:0.32516	validation_0-mlogloss:0.99540	validation_1-merror:0.36437	validation_1-mlogloss:1.03867
[20]	validation_0-merror:0.30428	validation_0-mlogloss:0.86571	validation_1-merror:0.34612	validation_1-mlogloss:0.93829
[30]	validation_0-merror:0.28998	validation_0-mlogloss:0.79365	validation_1-merror:0.33560	validation_1-mlogloss:0.88956
[40]	validation_0-merror:0.27614	validation_0-mlogloss:0.74631	validation_1-merror:0.33065	validation_1-mlogloss:0.86357
[50]	validation_0-merror:0.26353	validation_0-mlogloss:0.70940	validation_1-merror:0.32632	validation_1-mlogloss:0.84706
[60]	validation_0-merror:0.25046	validation_0-mlogloss:0.67646	validation_1-merror:0.32323	validation_1-mlogloss:0.83463
[70]	validation_0-merror:0.23832	validation_0-mlogloss:0.64854	validation_1-merror:0.32292	validation_1-mlogloss:0.82562
[80]	valid

2022-04-05 12:27:16,618 [INFO] #####training fold3#####


(12932, 16) (3233, 16)
[0]	validation_0-merror:0.34171	validation_0-mlogloss:1.32289	validation_1-merror:0.37210	validation_1-mlogloss:1.32654
[10]	validation_0-merror:0.31944	validation_0-mlogloss:0.99402	validation_1-merror:0.35354	validation_1-mlogloss:1.03196
[20]	validation_0-merror:0.30328	validation_0-mlogloss:0.86452	validation_1-merror:0.34457	validation_1-mlogloss:0.93126
[30]	validation_0-merror:0.28689	validation_0-mlogloss:0.79373	validation_1-merror:0.33591	validation_1-mlogloss:0.88613
[40]	validation_0-merror:0.27529	validation_0-mlogloss:0.74787	validation_1-merror:0.32849	validation_1-mlogloss:0.86215
[50]	validation_0-merror:0.26276	validation_0-mlogloss:0.71026	validation_1-merror:0.32354	validation_1-mlogloss:0.84650
[60]	validation_0-merror:0.24992	validation_0-mlogloss:0.68051	validation_1-merror:0.31983	validation_1-mlogloss:0.83593
[70]	validation_0-merror:0.24041	validation_0-mlogloss:0.65199	validation_1-merror:0.31488	validation_1-mlogloss:0.82624
[80]	valid

2022-04-05 12:27:18,837 [INFO] #####training fold4#####


(12932, 16) (3233, 16)
[0]	validation_0-merror:0.34929	validation_0-mlogloss:1.32322	validation_1-merror:0.37272	validation_1-mlogloss:1.32687
[10]	validation_0-merror:0.32331	validation_0-mlogloss:0.99698	validation_1-merror:0.35694	validation_1-mlogloss:1.03604
[20]	validation_0-merror:0.30854	validation_0-mlogloss:0.86818	validation_1-merror:0.34457	validation_1-mlogloss:0.93633
[30]	validation_0-merror:0.29145	validation_0-mlogloss:0.79415	validation_1-merror:0.33189	validation_1-mlogloss:0.88949
[40]	validation_0-merror:0.27675	validation_0-mlogloss:0.74589	validation_1-merror:0.32292	validation_1-mlogloss:0.86462
[50]	validation_0-merror:0.26353	validation_0-mlogloss:0.70842	validation_1-merror:0.31952	validation_1-mlogloss:0.85091
[60]	validation_0-merror:0.25325	validation_0-mlogloss:0.67799	validation_1-merror:0.31519	validation_1-mlogloss:0.84167
[70]	validation_0-merror:0.24064	validation_0-mlogloss:0.64614	validation_1-merror:0.31271	validation_1-mlogloss:0.83172
[80]	valid

In [27]:
df_submit

,sn,fault_time,label
0,000d33b21436,2020-09-02 16:42:54,3
1,005c5a9218ba,2020-06-28 19:05:16,1
2,0079283bde6e,2020-04-26 21:32:44,3
3,007bdf23b62f,2020-06-16 18:40:39,2
4,00a577a8e54f,2020-04-07 07:16:55,2
...,...,...,...
3006,ffbf46b4af21,2019-12-28 20:10:01,0
3007,ffc229b6cd9a,2020-06-27 02:39:08,0
3008,ffd44698a52b,2020-01-21 15:46:56,0
3009,fff73a9e5bd5,2020-03-01 22:43:43,0


In [21]:
csv_file = f"submit_{1}.csv"
df_submit.to_csv(csv_file, index=False)

from zipfile import ZipFile
zip_file = ZipFile(f"{csv_file}.zip", "w")
zip_file.write(csv_file )
zip_file.close()

logger.info("#####Well Done <_>#####")

2022-04-05 12:31:03,944 [INFO] #####Well Done <_>#####
